# Import Bibliotheken

In [2]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

OSError: [WinError 1455] Die Auslagerungsdatei ist zu klein, um diesen Vorgang durchzuführen. Error loading "c:\Users\flori\anaconda3\envs\ki_env\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

### Package Versionen

In [ ]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [ ]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [ ]:
comment_df = pd.read_excel('../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [ ]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [ ]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [ ]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [ ]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [ ]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [ ]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [ ]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [ ]:
# Länge des Trainungsets
len(train_set)

2798

In [ ]:
tokens = yield_tokens(train_set)

In [ ]:
next(iter(tokens))

['unfreundliches', 'personal']

In [ ]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [ ]:
len(vocab)

2383

In [ ]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 4, 0]

In [ ]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [ ]:
onehot = torch.zeros(1,len(vocab))

In [ ]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[33, 0, 46]

In [ ]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [ ]:
import torch
import torch.nn

In [ ]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [ ]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [ ]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

In [ ]:
count_list = ([ [l, flat_list.count(l)] for l in set(flat_list)])
count_list


[['Shop', 554],
 ['DigitalFueling', 30],
 ['Waschanlage', 462],
 ['Erscheinungsbild', 534],
 ['Luft', 86],
 ['Staubsauger', 32],
 ['Personal', 2790],
 ['WLAN', 3],
 ['Werkstatt', 45],
 ['Pricing', 1000],
 ['E-Mobilität', 7],
 ['Nacht-/Tankautomat', 95],
 ['E-Fuels', 1],
 ['AdBlue', 37],
 ['Kartenakzeptanzen', 145],
 ['Kraftstoffauswahl', 280],
 ['Öffnungszeiten', 129],
 ['Paketservice', 32],
 ['Tankpool', 3],
 ['Verkehrsanbindung', 111],
 ['SB-Waschboxen', 45],
 ['Sanitär', 138],
 ['Zapfsäulen', 45],
 ['Bistro', 716]]

In [ ]:
count_df = pd.DataFrame(count_list)
count_df.columns= ['Kategorie', 'Anzahl']
count_df

,Kategorie,Anzahl
0,Shop,554
1,DigitalFueling,30
2,Waschanlage,462
3,Erscheinungsbild,534
4,Luft,86
5,Staubsauger,32
6,Personal,2790
7,WLAN,3
8,Werkstatt,45
9,Pricing,1000


In [ ]:
kategorien_anzahl_bar = alt.Chart(count_df).mark_bar(color='#EC6500').encode(
    x = alt.X('Anzahl', title='Anzahl Vorkomnisse'),
    y = alt.Y('Kategorie', title='Kategorie', sort='-x')
).properties(
    title='Kategorieklassen'
)
kategorien_anzahl_text = kategorien_anzahl_bar.mark_text(
    align='left',
    baseline= 'middle',
    dx=3
).encode(
    text='Anzahl:Q'
)

kategorien_anzahl_chart = (kategorien_anzahl_bar + kategorien_anzahl_text)
kategorien_anzahl_chart.properties(width=550)

alt.LayerChart(...)

In [ ]:
cat_list = count_df[count_df['Anzahl'] >= 100]
cat_list = cat_list['Kategorie'].values.tolist()

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [ ]:
flat_set = set(cat_list)
flat_set

{'Bistro',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Personal',
 'Pricing',
 'Sanitär',
 'Shop',
 'Verkehrsanbindung',
 'Waschanlage',
 'Öffnungszeiten'}

In [ ]:
unflat = [[x] for x in flat_set]

In [ ]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1,)

label_vocab.set_default_index(vocab["<unk>"])

In [ ]:
label_vocab.lookup_token(3)

'Kraftstoffauswahl'

#### Dictionary erstellen

In [ ]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Shop': 0,
 'Verkehrsanbindung': 1,
 'Personal': 2,
 'Waschanlage': 3,
 'Kartenakzeptanzen': 4,
 'Pricing': 5,
 'Erscheinungsbild': 6,
 'Kraftstoffauswahl': 7,
 'Öffnungszeiten': 8,
 'Sanitär': 9,
 'Bistro': 10}

#### Tensor

In [ ]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [ ]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[2, 3]

In [ ]:
labels = torch.LongTensor(pos_vals)
labels

tensor([2, 3])

In [ ]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [ ]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [ ]:
# label_pipeline = lambda x: int(x) -1

In [ ]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [ ]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [ ]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [ ]:
vocab_size = len(vocab)
num_class = flat_set.__len__()
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [ ]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.03

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Fehlerfunktion
bce_loss = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 150

In [ ]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold= 0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [ ]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])
tensor([1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0])
tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0])
tensor([1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0])
tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 1, 0, 0, 1, 0,

In [ ]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

KeyboardInterrupt: 

In [ ]:
batch_train_accuracy

tensor(0.4439)

### Auswertung Fehlerentwicklung

In [ ]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.678934
1,1,0.674037
2,2,0.673605
3,3,0.673448
4,4,0.673375


In [ ]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
).properties(title='Trainingsfehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [ ]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.314868
1,1,0.358113
2,2,0.360615
3,3,0.362044
4,4,0.363117


In [ ]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
).properties(title='Trainingsgenauigkeit in Abhängigkeit von der Anzahl der Epochen')
accuracy_chart

alt.Chart(...)

## Evaluierung

In [ ]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2439, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=11, bias=True)
)

### Gesamtevaluierung

In [ ]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [ ]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [ ]:
total_valid_accuracy

tensor(0.3945)

### Evaluierung nach Klassen

In [ ]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=flat_set.__len__()
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [ ]:
total_valid_accuracy

tensor([0.5680, 0.8674, 0.9625, 0.9349, 0.8616, 0.8490, 0.9616, 0.8557, 0.9758,
        0.8799, 0.9641])

### Evaluierung nach der Anzahl der Epochen

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [ ]:
test_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [ ]:
vocab_size = len(vocab)
num_class = flat_set.__len__()
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [ ]:
bce_loss = nn.BCEWithLogitsLoss()

In [ ]:
learning_rate = 0.03

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [ ]:
num_epochs = 150

In [ ]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [ ]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = bce_loss(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()



Training Accuracy for epoch 0: 0.3191565275192261
Validation Accuracy for epoch 0: 0.36240172386169434
Training Accuracy for epoch 1: 0.3573981523513794
Validation Accuracy for epoch 1: 0.3981415331363678
Training Accuracy for epoch 2: 0.3981415331363678
Validation Accuracy for epoch 2: 0.4110078513622284
Training Accuracy for epoch 3: 0.4167262315750122
Validation Accuracy for epoch 3: 0.4177984297275543
Training Accuracy for epoch 4: 0.4213724136352539
Validation Accuracy for epoch 4: 0.42530378699302673
Training Accuracy for epoch 5: 0.4242315888404846
Validation Accuracy for epoch 5: 0.42530378699302673
Training Accuracy for epoch 6: 0.4231593906879425
Validation Accuracy for epoch 6: 0.4278055727481842
Training Accuracy for epoch 7: 0.42637598514556885
Validation Accuracy for epoch 7: 0.42637598514556885
Training Accuracy for epoch 8: 0.4285203814506531
Validation Accuracy for epoch 8: 0.4295925796031952
Training Accuracy for epoch 9: 0.4285203814506531
Validation Accuracy for epo

In [ ]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "validation"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [ ]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
 x='Epoch',
 y='Loss',
 color='Type'
).properties(title='Fehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_eval_loss_chart

alt.Chart(...)

In [ ]:
valid_accuracy_hist

{0: array(0.2873481, dtype=float32),
 1: array(0.30164403, dtype=float32),
 2: array(0.31629735, dtype=float32),
 3: array(0.3062902, dtype=float32),
 4: array(0.31844175, dtype=float32),
 5: array(0.37848464, dtype=float32),
 6: array(0.37776983, dtype=float32),
 7: array(0.3856326, dtype=float32),
 8: array(0.3817012, dtype=float32),
 9: array(0.3849178, dtype=float32),
 10: array(0.3849178, dtype=float32),
 11: array(0.38741958, dtype=float32),
 12: array(0.38599, dtype=float32),
 13: array(0.39170837, dtype=float32),
 14: array(0.39563975, dtype=float32),
 15: array(0.39599714, dtype=float32),
 16: array(0.39206576, dtype=float32),
 17: array(0.39742672, dtype=float32),
 18: array(0.39742672, dtype=float32),
 19: array(0.39706933, dtype=float32),
 20: array(0.39528236, dtype=float32),
 21: array(0.39814153, dtype=float32),
 22: array(0.39599714, dtype=float32),
 23: array(0.39063618, dtype=float32),
 24: array(0.39027876, dtype=float32),
 25: array(0.39528236, dtype=float32),
 26: 

In [ ]:
train_accuracy_hist

{0: array([0.57214344, 0.82402   , 0.9958299 , 0.99916595, 0.99666387,
        0.85571307, 0.9591326 , 0.93744785, 0.97581315, 0.97331107,
        0.98999166, 0.8615513 , 0.8098415 , 0.98415345, 0.9641368 ,
        0.8790659 , 0.9891576 , 0.99916595, 0.96330273,        nan,
        0.8932444 , 0.9916597 , 0.9849875 , 0.97497916], dtype=float32),
 1: array(0.2873481, dtype=float32),
 2: array(0.30164403, dtype=float32),
 3: array(0.31629735, dtype=float32),
 4: array(0.3062902, dtype=float32),
 5: array(0.31844175, dtype=float32),
 6: array(0.37848464, dtype=float32),
 7: array(0.37776983, dtype=float32),
 8: array(0.3856326, dtype=float32),
 9: array(0.3817012, dtype=float32),
 10: array(0.3849178, dtype=float32),
 11: array(0.3849178, dtype=float32),
 12: array(0.38741958, dtype=float32),
 13: array(0.38599, dtype=float32),
 14: array(0.39170837, dtype=float32),
 15: array(0.39563975, dtype=float32),
 16: array(0.39599714, dtype=float32),
 17: array(0.39206576, dtype=float32),
 18: ar

In [ ]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [ ]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "validation"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [ ]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.287348,train
1,2,0.301644,train
2,3,0.316297,train
3,4,0.306290,train
4,5,0.318442,train
...,...,...,...
144,145,0.399929,validation
145,146,0.399929,validation
146,147,0.399929,validation
147,148,0.399214,validation


In [ ]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
 x='Epoch',
 y='Accuracy',
 color='Type'
).properties(title='Genauigkeiten des Trainings- und Testdatensatzes nach der Anzahl der Epochen')
train_eval_accuracy_chart

alt.Chart(...)